#### Modeling your NoSQL database or Apache Cassandra database

1. Design tables to answer the queries outlined in the project template

2. Write Apache Cassandra CREATE KEYSPACE and SET KEYSPACE statements

3. Develop your CREATE statement for each of the tables to address each question

4. Load the data with INSERT statement for each of the tables

5. Include IF NOT EXISTS clauses in your CREATE statements to create tables only if the tables do not already exist. We recommend you also include DROP TABLE statement for each table, this way you can run drop and create tables whenever you want to reset your database and test your ETL pipeline

6. Test by running the proper select statements with the correct WHERE clause

#### Build ETL Pipeline

1. Implement the logic in section Part I of the notebook template to iterate through each event file in event_data to process and create a new CSV file in Python

2. Make necessary edits to Part II of the notebook template to include Apache Cassandra CREATE and INSERT statements to load processed records into relevant tables in your data model

3. Test by running SELECT statements after running the queries on your database

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    #print(files)
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum (1 for line in f))

In [ ]:
import pandas as pd

df = pd.read_csv('event_datafile_new.csv')
df.head()

#### Begin writing your Apache Cassandra code in the cells below¶

Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS semispaqify
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1}"""
    )
except Exception as e:
    print(e)

Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('semispaqify')
except Exception as e:
    print(e)

Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Query 1

In [ ]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Creating the tables
query = 'CREATE TABLE IF NOT EXISTS song_session_tbl'
query = query + "(artist_name text, song_title text, song_length float, session_id int, \
item_session int, PRIMARY KEY (session_id, item_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_session_tbl (artist_name, song_title, song_length, session_id, item_session)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        
        # I did typecasting to match the table model
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3]))) 

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

query = "SELECT artist_name, song_title, song_length FROM song_session_tbl WHERE session_id = 338 AND item_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

#### Query 2

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = 'CREATE TABLE IF NOT EXISTS users_tbl'
query = query + "(artist_name text, song_title text, user_firstname text, user_lastname text, \
user_id int, session_id int, item_session int, PRIMARY KEY ((user_id, session_id), item_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)         

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_tbl (artist_name, c, user_firstname, user_lastname, \
user_id, session_id, item_session)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        
        # I did typecasting to match the table model
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3]))) 

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

query = "SELECT artist_name, song_title, user_firstname, user_lastname FROM users_tbl \
WHERE user_id = 10 AND session_id = 182 ORDER BY item_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.user_firstname, row.user_lastname)

#### Query 3

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
   
query = 'CREATE TABLE IF NOT EXISTS user_song_tbl'
query = query + "(user_firstname text, user_lastname text, song_title text, PRIMARY KEY (song_title, user_lastname))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song_tbl (user_firstname, user_lastname, song_title)"
        query = query + "VALUES (%s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        
        # I did typecasting to match the table model
        session.execute(query, (line[1], line[4], line[9])) 

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

query = "SELECT user_firstname, user_lastname FROM user_song_tbl WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user_firstname, row.user_lastname)

#### Drop tables

In [ ]:
query = "DROP TABLE song_session_tbl"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query1 = "DROP TABLE users_tbl"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query2 = "DROP TABLE user_song_tbl"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

#### Shut sessions

In [ ]:
session.shutdown()
cluster.shutdown()